## Midterm Project

# Analysis of Breweries


__Data Source__
         
   https://raw.githubusercontent.com/openbrewerydb/openbrewerydb/master/breweries.csv
   
   Data file size: 1.5 MB
 
This is the open-source dataset for the Open Brewery DB API which is served by a REST API built with Ruby on Rails. Retrieved from https://github.com/openbrewerydb/openbrewerydb

This dataset represents the data about breweries in the United states, Ireland and United Kingdom. It has 7984 rows and 16 columns. Some of the columns are brewery id, name, brewery type, street address, city, state, country, website URL. I am interested in this data because I like Exploring beers. It has a huge diversity of beer styles, taste profiles and in the future as a business opportunity I would like to invest in this industry.

__Introduction__

Beer is an alcoholic beverage produced by extracting raw materials with water, boiling with hops, and fermenting it. Beer brewing procedure has a cultural history behind it. In the 21st century the brewing industry has become large-scale with the use of technology. Brewery industry has grown so much that it has a significant contribution to the economic growth of the nation. America’s beer industry supports more than two million jobs, provides more than 102 billion dollor in wages and benefits, and contributes more than 331 billion dollor to the U.S. economy. These stats made me choose this dataset for the analysis. In this project I will try to get a generalized idea of how the brewery industry is distributed over the USA based on the given data.

There are distinct beer industry market segments.

- Large-  A brewery that produces Approximately 2 million barrels of beer per year.

- Regional-  A brewery with an annual beer production of between 15,000 and 2,000,000 barrels.

- Microbrewery- A brewery that produces less than 15,000 barrels of beer per year and sells 75 percent or more of its beer off-site.

- Brewpub-  A restaurant-brewer that sells 25 percent or more of its beer on-site and operates significant food services.

- Taproom-  A professional brewery that sells 25 percent or more of its beer on-site and does not operate significant food services.The beer is brewed primarily for sale in the taproom, and is often dispensed directly from the brewery’s storage tanks.

- Contract-  A business that hires another brewery to produce its beer.

- Proprietary-  A licensed tenant brewery that physically takes possession of a shared brewery while brewing.

- Nano-  A brewery which produces at least 1,000 barrels per year.(1/3 to 1/10th the size of a microbrewery)

__Objectives__

- To discover which type of brewery is popular
- To discover which location in the USA has potential for the brewery industry.
- To understand the overall industry insights.

In [ ]:
import requests
from io import StringIO 
import pandas as pd
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# Converting .csv file into pandas dataframe and adding index column.

brewery_df = pd.read_csv('https://raw.githubusercontent.com/openbrewerydb/openbrewerydb/master/breweries.csv',
                 index_col= 0).reset_index()

#### Understanding the dataset

In [ ]:
# Checking the characteristics of the data.

brewery_df.head(5)

In [ ]:
# Number of rows and columns in dataframe.

brewery_df.shape

In [ ]:
# Column names and their data types. 

brewery_df.info()

The Dataset has 16 columns and 7984 rows in it. Each column has data types which are objects and floats. There are inconsistencies in the data which are mentioned below.

#### Inconsistency in data
- There are many columns( address_2, address_3,county_province,tags ) which don’t have significant entries in it. And   that information can also be retrieved from other columns(address can be located by postal_code).
- Missing values of latitude and longitude for 2829 number of records.
- Missing values for website_url for 1190 rows.
- Missing values of the state column for 142 entries.
- Missing values of the phone column for 810 entries.
- Data type of all the columns is the object.

#### Cleaning the data

To make the dataset more efficient for our analysis we need to perform some operations. In the following cells some of the data cleaning operations are done at this initial stage.

In [ ]:
# Converting the data type base on the contents of the column.

brewery_df=brewery_df.convert_dtypes()
brewery_df.dtypes

In [ ]:
# While trying to convert the data type of the phone column to integer got an error that it contains'3.53599E+11'
# which is not an int.
# replacing non int value with '0'.

brewery_df['phone'] = pd.to_numeric(brewery_df.phone.astype(str),errors='coerce').fillna(0).astype(int)
  
brewery_df.dtypes

In [ ]:
# Dropping address_2, address_3 and country_province because there are very few entries for them and 
# we have street and postal_code information to get the location.

brewery_df = brewery_df.drop(['address_2'], axis=1)
brewery_df = brewery_df.drop(['address_3'], axis=1)
brewery_df = brewery_df.drop(['county_province'], axis=1)

# Dropping tags Column as it is empty.

brewery_df = brewery_df.drop(['tags'], axis=1)

brewery_df.shape

In [ ]:
#Selecting random columns for checking the consistency in the dataframe.

brewery_df.sample(5)

__Note :__ Since data which is missing in the columns are closely related to brewery ID and they all are unique(like phone number , lat lon values). So filling the values for missing data is not suitable for this data set. It will be more effective to proceed as it is.

In [ ]:
# Checking the types of breweries.

brewery_df['brewery_type'].unique()

__Note :__  Breweries are the places where beer is produced and sold on-site or off-site.There is no production of beer in the bar. Bar is the place where different types of alcohol are served. So there are 2 entries in the dataset as type bar which are outliers which need to be removed from the dataset.

In [ ]:
# Dropping columns where brewery_type is bar.

brewery_df.drop(brewery_df[brewery_df['brewery_type']== 'bar'].index, inplace = True)
brewery_df['brewery_type'].value_counts()

In [ ]:
brewery_df.shape

__Note :__ Original size of the dataset was (7984, 16). After dropping the rows and columns it is (7982,12).

In [ ]:
# Converting the data type of brewery_type column into category.

brewery_df['brewery_type'] = brewery_df['brewery_type'].astype('category')
brewery_df.dtypes

In [ ]:
# Checking the number of countries.

brewery_df['country'].value_counts()

#### Sanity check on categorical data

- Checking for the outliers.
- Checking the distribution of data (frequent values in the column).
- Checking why the values are repeated to verify if there are duplicate entries.

In [ ]:
# Checking the overall distribution of the data in columns with dtype string.

brewery_df.describe(include='string')

__Note :__ obdb_id has no repeating values in the column. That means each entry can uniquely identify the row.

In [ ]:
# Most repeated brewery name.

brewery_df[brewery_df['name'] == 'Ballast Point Brewing Company'].dropna(how='all')


In [ ]:
# Most repeated streets in dataset.

brewery_df[brewery_df['street'] == '303 Main St'].dropna(how='all')

__Note :__ Most frequent street in the dataset is 303 Main St. Which might mislead us thinking that 303 Main St is known as a brewery area. But when we look at all other information all three breweries are located in different states which have 303 Main street.

To find in which area breweries are most clustered we need to look at frequent postal codes.

In [ ]:
# Most repeated posta_code in the dataset.

brewery_df[brewery_df['postal_code'] == '97701'].dropna(how='all')

In [ ]:
# Most repeated city.

brewery_df['city'].value_counts().head(5)

In [ ]:
# Creating new dataframe where city name is Portland.

df_portland = brewery_df[brewery_df['city'] == 'Portland'].dropna(how='all')

#Ploting the graph of top 5 brewery types in Portland.

df_portland['brewery_type'].value_counts()[:5].plot(kind='bar',color='#CF686D')
plt.title(" Types of breweries in Portland",fontweight="bold")
plt.xlabel('Types of breweries',fontsize=13)
plt.ylabel('Frequency',fontsize=13)
plt.show()

__Observation__: Portland has a higher number of micro and brewpub types of breweries than other brewery types.

In [ ]:
# Creating new dataframe where brewery type is closed.

df_type_closed= brewery_df[brewery_df['brewery_type'] == 'closed'].dropna(how='all')

#Ploting the graph of cities which have the most number of closed breweries.

df_type_closed['city'].value_counts()[:10].plot(kind='bar',color='#CF686D')
plt.title("Closed brewries in City",fontweight="bold")
plt.xlabel('City',fontsize=13)
plt.ylabel('Frequency',fontsize=13)
plt.show()

__Observation :__ There are not a significant number of breweries which are closed in particular cities or states.

In [ ]:
# Creating new dataframe where brewery type is large.

df_type_large= brewery_df[brewery_df['brewery_type'] == 'large'].dropna(how='all')

# Creating new dataframe where brewery type is micro.

df_type_micro = brewery_df[brewery_df['brewery_type'] == 'micro'].dropna(how='all')

#Ploting the graph of cities which have large breweries.

fig= plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
df_type_large['city'].value_counts()[:5].plot(kind='bar',color='#CF686D')
plt.title("Large brewries in City",fontweight="bold")
plt.xlabel('City',fontsize=13)
plt.ylabel('Frequency',fontsize=13)


#Ploting the graph of cities which have micro breweries.

plt.subplot(1,2,2)
df_type_micro['city'].value_counts()[:5].plot(kind='bar',color='#CF686D')
plt.title(" Micro breweries in City",fontweight="bold")
plt.xlabel('City',fontsize=13)
plt.ylabel('Frequency',fontsize=13)
plt.show()

plt.show()

__Observation :__ San Diego has 7 large breweries. We can not see Portland in the top 10 cities even though it has the highest number of breweries. Because Portland is famous for micro breweries.

In [ ]:
#Ploting the graph of top 5 states which have large breweries.

df_type_large['state'].value_counts()[:5].plot(kind='bar',color='#CF686D')

plt.title("Large brewries in State",fontweight="bold")
plt.xlabel('City',fontsize=13)
plt.ylabel('Frequency',fontsize=13)
plt.show()

__Observation :__ Along with West coast states Wisconsin and Virginia also has a significant number of large breweries.

In [ ]:
# Creating new dataframe with breweries in Ireland and UK.

 
df_Ire_UK= brewery_df.loc[(brewery_df['country'] == 'Ireland') | 
                         (brewery_df['country'] == 'England') | 
                         (brewery_df['country'] == 'Scotland')]
df_Ire_UK.info()

__Note :__ All the results above have US data. Because the number of entries of US data is very large as compared to other countries. Created a new data frame to see if we can find some interesting trends for Ireland, England and Scotland breweries.

In [ ]:
# Number of breweries in each country.

df_Ire_UK['country'].value_counts()

In [ ]:
df_Ire_UK.describe(include= 'all')

__Observation :__ In this dataframe also there are similar trends like these countries also have mostly micro type of breweries. Dublin is the city with the highest number of breweries there.

In [ ]:
!pip install chart_studio

In [ ]:
import chart_studio.plotly as py 
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

#Creating the new dataframe with breweries only in the US. 

df_US = brewery_df[brewery_df['country'] == 'United States']
df_US.info()

In [ ]:
#Creating new column as brewery_count which will calculate and store the breweries in every state.

df_US['brewery_count'] = df_US['obdb_id'].groupby(df_US['state']).transform('count')
df_US.head(2)


In [ ]:
# Extract the entries which have latitude and longitude. 

df_latlon=df_US[df_US.longitude.notnull()]

In [ ]:
# Creating a new column which will store the information displayed
# while hovering on the interactive graph. 

df_latlon['text_template'] = df_latlon[['name', 'city','state']].agg(','.join, axis=1)
df_latlon.head(2)

In [ ]:
# Plotting the interactive map which will show the location of breweries in the US.

data = go.Scattergeo(
        lon = df_latlon['longitude'],
        lat = df_latlon['latitude'],
        text = df_latlon['text_template'],
        hoverinfo="text",
        mode = 'markers',
        marker = dict(symbol = 'star',size=4,colorscale = 'matter',colorbar = {'title':"No of Brewries per State"} ),
        marker_color = df_latlon['brewery_count'],
        )
layout = dict(title = 'Breweries locations USA',
              geo_scope = 'usa'
             )

choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)

In [ ]:
# Since the lat, lon values are missing for almost 2000 breweries 
# will try plotting graph with states for cross verification.

df_US['state'].value_counts()[:15].plot(kind='bar',color='#CF686D')

plt.title("Brewries in State",fontweight="bold")
plt.xlabel('State',fontsize=13)
plt.ylabel('Frequency',fontsize=13)
plt.show()



__Observation :__ Even when we plot graph with all the breweries state wise. It is showing a similar result as in lat lon map. 

### What we learned ?

1] Ballast Point Brewing Company has the highest number of franchisees in craft breweries. Mostly they are in California.

2] In Bend, Oregon there is a hub of breweries. There are large and regional breweries located in the postal code – 97701 area.

3] Portland, Denver, San Diego, Seattle and Chicago are the top five cities which have the highest number of breweries.

4] Portland has the greatest number of micro and brewpub types of breweries. Further analysis found, since the 80s, Portland is pioneering the craft beer movement with fresh ingredients like Willamette Valley hops and Bull Run water. City has a nickname “Beervana” due to its vast range of beer industry.

5] There is a correlation between the number of breweries and states. We can see that mostly breweries are popular in states California, Washington, Colorado, New York and Michigan.

6] Profitable locations of breweries defer based on which type of beer you want to make. Micro-breweries mostly make craft beers which they sell on-site also, whereas large breweries rarely sell cell beers on-site. San Diego is good for large breweries and Portland is a good choice for micro-breweries.